# Atomic-Force-Microscopy (AFM) - Interactive plot  with Matplotlib

## Description
Plot AFM-data as a 2D image, 3D surface and as a profile along a straight line. The line profile can be created interactively with ipython-widgtes.

<img src="./AFM.jpg">

## TODO

X- and Y-Coordiantes are in data-coordinates 

## Conclusion
This is a proof of concept. It works, but the matplotlib 3D-backend might be too slow.


## 0. Imports

In [11]:
%matplotlib tk
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec

## 1. Load Data

- The example-data was cleaned up in Gwyddion and exported as an ASCII-file with heder
- It might be possible to load the data directly, but you'll want to clean it up with gwyddion anyway.

In [12]:
filepath = "./04181616.txt"

#--- numpy import works directly with ascii-files ---
data = np.loadtxt(filepath)

#--- this is only for the header ---
headerlines = []
with open(filepath, "r") as f:
    lines = f.readlines()
    
for line in lines:
    if line.startswith("#"):
        headerlines.append(line)

#--- print results
print("   Header    ")
print("-------------")
for h in headerlines:
    print(h)

print("   data-shape   ")
print("----------------")
print(np.shape(data))

   Header    
-------------
# Kanal: Height

# Breite: 1.000 µm

# Höhe: 1.000 µm

# Value units: m

   data-shape   
----------------
(512, 512)


In [13]:
def get_line(x1, x2, y1, y2):
    """ 
    Find integers lying on a line. Used to find valid indices in a 2d data-array that intersect
    with the line
    
    Code from:
    http://stackoverflow.com/questions/25837544/get-all-points-of-a-straight-line-in-python
    
    x1,x2,y1,y2 : coordinates of start- and endpoint of the line
    """
    points = []
    issteep = abs(y2-y1) > abs(x2-x1)
    if issteep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    rev = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        rev = True
    deltax = x2 - x1
    deltay = abs(y2-y1)
    error = int(deltax / 2)
    y = y1
    ystep = None
    if y1 < y2:
        ystep = 1
    else:
        ystep = -1
    for x in range(x1, x2 + 1):
        if issteep:
            points.append((y, x))
        else:
            points.append((x, y))
        error -= deltay
        if error < 0:
            y += ystep
            error += deltax
    # Reverse the list if the coordinates were reversed
    if rev:
        points.reverse()
        
    return points


def make_plot(data, linecoords):
    """
    data : 2D numpy-array
    linecoords : x0,x1,y0,y1 --> arguments for get_line()
    """
    
    # flip data from row-colummn to x-y
    d = np.flipud(data)
    points = get_line(*linecoords)
    
    lx = []
    ly = []
    lv = []
    for p in points:
        lx.append(p[0])
        ly.append(p[1])
        lv.append(d[p[1],p[0]])

    #--- setup plot, layout and colormap
    plt.close("all")
    fig = plt.figure()

    gs = gridspec.GridSpec(2, 2,
                          width_ratios=[1,1])
    ax1 = fig.add_subplot(gs[0:,0], projection="3d")
    ax2 = fig.add_subplot(gs[0,1])
    ax3 = fig.add_subplot(gs[1,1])

    cmap = cm.gist_earth

    #--- 3D raw data 
    #------ generating x and y vectors from xlims/ylims
    dx,dy = np.shape(data)
    xvec, yvec = np.arange(0,dx), np.arange(0,dy)
    x,y = np.meshgrid(xvec, yvec)
    z = d
    ax1.plot_surface(x,y,z, linewidth=0, cmap=cmap)
    ax1.plot(lx,ly,lv, color="red", alpha=0.7)

    #--- 2D image
    vmin, vmax = np.min(z), np.max(z)
    im = ax2.imshow(z, origin="upper", cmap=cmap, vmin=vmin, vmax=vmax)
    ax2.plot(lx,ly,color="red", alpha=0.7)
    cbar = fig.colorbar(im, ax=ax2, label=zlabel)

    #--- Linegraph
    ax3.plot(lv, color="red", alpha=1)
    ax3.plot(lv, marker="x", ls="", color="red", alpha=0.2)
    
    for ax in [ax1,ax2]:
        ax.set_xlim(0,dx)
        ax.set_ylim(0,dy)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
    ax1.set_zlabel(zlabel)
    ax3.set_ylabel(zlabel)
        
    #--- finalize and show
    fig.suptitle("height")
    fig.canvas.draw()

# Setup plot

- axis labels. (TODO: xlims, ylims)
- As the header shows, the data is in m --> convert to nm
    - You know your data (if not, see the header)


In [14]:
#--- convert to nanometer
plotdata = data*1E9

#--- these are used for plotting; defined as globals(not function arguments)
xlabel = r"data coord."
ylabel = r"data coord."
zlabel = r"$nm$"

# Plot
- Change intersection interactively; press button to refresh plot (set __manual=False to update in realtime)
- Data is not interpolated. The line connects real data points.

In [15]:
from ipywidgets import widgets, interact

dx,dy = np.shape(data)

#--- wrap plotting function to make plotdata-argument static
make_plot_wrapped = lambda x0,x1,y0,y1: make_plot(plotdata,[x0,x1,y0,y1])

interact(make_plot_wrapped, x0=[0,dx-1], x1=[0,dx-1], 
         y0=[0,dy-1], y1=[0,dy-1], __manual=True)

<function __main__.<lambda>>